# Szyfry blokowe - ECB, OCB, CTR
---

### Imports

In [1]:
import random
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Util.Counter import new
from Crypto.Random import get_random_bytes
from time import time

---
### Data

In [24]:
def random_data(size):
    characters = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
    random_indices = [random.randint(0, len(characters) - 1) for _ in range(size)]
    return ''.join([characters[i] for i in random_indices])

sizes = [100000, 1000000, 1000000000]

for size in sizes:
    file_name = 'data/data_' + str(size) + ".txt"
    data = random_data(size)
    with open(file_name, 'w') as file:
        file.write(data)
        

In [2]:
sizes = [100000, 1000000, 1000000000]
data = []

for size in sizes:
    file_name = 'data/data_' + str(size) + ".txt"
    with open(file_name, 'r') as file:
        data.append(file.read())

In [3]:
modes = [
    (AES.MODE_ECB, 'ECB'),
    (AES.MODE_OCB, 'OCB'),
    (AES.MODE_CTR, 'CTR')
]

---
### Speed test

In [4]:
def printTime(mode, size, time, type = 'Encryption'):
    print(f"{type} {mode} mode with {size} bytes: {time} seconds")

def _chunks(data, size=16):
    return [data[i:i + size] for i in range(0, len(data), size)]

In [5]:
def modeEBC(data, size, key, time_print = True, data_print = False, make_mistake = False):
    cipher = AES.new(key, AES.MODE_ECB)
    padded_data = pad(data.encode(), AES.block_size)
    start = time()
    encrypted = cipher.encrypt(padded_data)
    end = time()

    if make_mistake:
        encrypted = bytearray(encrypted)
        encrypted[1] = encrypted[1] ^ 1
        encrypted = bytes(encrypted)
        
    if time_print:
        printTime("ECB", size, end - start)
    start = time()
    decrypted = cipher.decrypt(encrypted)
    end = time()
    if time_print:
        printTime("ECB", size, end - start, 'Decryption')

    if data_print:
        print(f"Data: {data}")
        print(f"Encrypted: {encrypted.decode('latin-1')}")
        print(f"Decrypted: {decrypted.decode('latin-1')}")

In [6]:
def modeOCB(data, size, key, time_print = True, data_print = False, make_mistake = False):
    nonce = get_random_bytes(15)

    cipher_text = b''
    
    start = time()
    cipher = AES.new(key, AES.MODE_OCB, nonce = nonce)
    cipher_text, tag = cipher.encrypt_and_digest(data.encode())
    end = time()

    if make_mistake:
        cipher_text = bytearray(cipher_text)
        cipher_text[1] = cipher_text[1] ^ 1
        cipher_text = bytes(cipher_text)

    if time_print:
        printTime("OCB", size, end - start)

    plain_text = b''
    
    try:
        start = time()
        cipher = AES.new(key, AES.MODE_OCB, nonce = nonce)
        plain_text = cipher.decrypt_and_verify(cipher_text, tag)
        end = time()
    except ValueError:
        start = time()
        cipher = AES.new(key, AES.MODE_OCB, nonce = nonce)
        plain_text = cipher.decrypt(cipher_text)
        end = time()

    if time_print:
        printTime("OCB", size, end - start, 'Decryption')
        
    if data_print:
        print(f"Original data: {data}")
        print(f"Encrypted data: {cipher_text.decode('latin-1')}")
        print(f"Decrypted data: {plain_text.decode('latin-1')}")


In [7]:
def modeCTR(data, size, key, time_print = True, data_print = False, make_mistake = False):
    counter = new(128)

    cipher = AES.new(key, AES.MODE_CTR, counter = counter)

    start = time()
    encrypted = cipher.encrypt(data.encode())
    end = time()

    if make_mistake:
        encrypted = bytearray(encrypted)
        encrypted[1] = encrypted[1] ^ 1
        encrypted = bytes(encrypted)

    if time_print:
        printTime("CTR", size, end - start)
    
    cipher = AES.new(key, AES.MODE_CTR, counter = counter)

    start = time()
    decrypted = cipher.decrypt(encrypted)
    end = time()

    if time_print:
        printTime("CTR", size, end - start, 'Decryption')

    if data_print:
        print(f"Original data: {data}")
        print(f"Encrypted data: {encrypted.decode('latin-1')}")
        print(f"Decrypted data: {decrypted.decode('latin-1')}")
    

In [8]:
key = get_random_bytes(16)
for mode in modes:
    cipher_encrypt = AES.new(key, mode[0])
    cipher_decrypt = AES.new(key, mode[0])
    print('Mode: ' + mode[1])
    for i in range(3):
        if mode[0] == AES.MODE_ECB:
            modeEBC(data[i], sizes[i], key)
        if mode[0] == AES.MODE_OCB:
            modeOCB(data[i], sizes[i], key)
        if mode[0] == AES.MODE_CTR:
            modeCTR(data[i], sizes[i], key)
    print()

Mode: ECB
Encryption ECB mode with 100000 bytes: 0.0010020732879638672 seconds
Decryption ECB mode with 100000 bytes: 0.0 seconds
Encryption ECB mode with 1000000 bytes: 0.0009970664978027344 seconds
Decryption ECB mode with 1000000 bytes: 0.0010075569152832031 seconds
Encryption ECB mode with 1000000000 bytes: 0.5479245185852051 seconds
Decryption ECB mode with 1000000000 bytes: 0.632866382598877 seconds

Mode: OCB
Encryption OCB mode with 100000 bytes: 0.0010371208190917969 seconds
Decryption OCB mode with 100000 bytes: 0.0009970664978027344 seconds
Encryption OCB mode with 1000000 bytes: 0.003964900970458984 seconds
Decryption OCB mode with 1000000 bytes: 0.004002809524536133 seconds
Encryption OCB mode with 1000000000 bytes: 5.446765661239624 seconds
Decryption OCB mode with 1000000000 bytes: 5.437963008880615 seconds

Mode: CTR
Encryption CTR mode with 100000 bytes: 0.0 seconds
Decryption CTR mode with 100000 bytes: 0.0 seconds
Encryption CTR mode with 1000000 bytes: 0.00400209426

---
### Mistake propagation

In [12]:
for mode in modes:
    text = 'Propapagacja bledu w trybie ' + mode[1]
    if mode[0] == AES.MODE_ECB:
        modeEBC(text, len(text), key, False, True, True)
    if mode[0] == AES.MODE_OCB:
        modeOCB(text, len(text), key, False, True, True)
    if mode[0] == AES.MODE_CTR:
        modeCTR(text, len(text), key, False, True, True)
    print()
    

Data: Propapagacja bledu w trybie ECB
Encrypted: ÕmËR]gü÷9Òzåÿ²ñÜ Nµ¾~lÅõ·
Decrypted: #Ùwú}Ìý?S­ddu w trybie ECB

Original data: Propapagacja bledu w trybie OCB
Encrypted data: iú,ÎRò	ROYçXlâ£ã(Þ¦%r¿~Ê!cbÙ
Decrypted data: ñÄE£ä@ï;½Ýée?

Original data: Propapagacja bledu w trybie CTR
Encrypted data: õó¹G=muÝ§2½é6Û½#únF¼|«Õö
Decrypted data: Psopapagacja bledu w trybie CTR



---
### CBC implementation using ECB

In [10]:
def modeCBC(data, key):
    
    iv = get_random_bytes(16)

    previous_block = iv
    cipher_text = b''

    for block in _chunks(data):
        cipher = AES.new(key, AES.MODE_ECB)
        block = pad(block.encode(), AES.block_size)
        cipher_block = cipher.encrypt(bytes([a ^ b for a, b in zip(block, previous_block)]))
        cipher_text += cipher_block
        previous_block = cipher_block

    plain_text = b''
    previous_block = iv
    for block in _chunks(cipher_text):
        cipher = AES.new(key, AES.MODE_ECB)
        decrypted_block = cipher.decrypt(block)
        plain_text += bytes([a ^ b for a, b in zip(decrypted_block, previous_block)])
        previous_block = block

    print(cipher_text.decode('latin-1'))
    print(plain_text.decode('latin-1'))

modeCBC("Testowanie trybu CBC", key)

Pû$;c£SÚ¶:Pî	Â· \
Testowanie trybu CBC
